In [1]:
import sopa
import spatialdata


sdata = spatialdata.read_zarr("/Volumes/ProstateCancerEvoMain/dbs/Ongoing/Region2/ARMS/Region2_FULL_TilesAdded.CellsAdded.zarr")
sdata

/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'rea

SpatialData object, with associated Zarr store: /Volumes/ProstateCancerEvoMain/dbs/Ongoing/Region2/ARMS/Region2_FULL_TilesAdded.CellsAdded.zarr
├── Images
│     └── '_20221020095443.ome': DataTree[cyx] (3, 114243, 79178), (3, 28560, 19794), (3, 7140, 4948), (3, 1785, 1237), (3, 446, 309)
├── Points
│     └── 'ARMS_Marks': DataFrame with shape: (<Delayed>, 2) (2D points)
└── Shapes
      ├── 'cell_boundaries': GeoDataFrame shape: (299769, 1) (2D shapes)
      └── 'tiles': GeoDataFrame shape: (2109, 4) (2D shapes)
with coordinate systems:
    ▸ 'aligned', with elements:
        _20221020095443.ome (Images), cell_boundaries (Shapes), tiles (Shapes)
    ▸ 'global', with elements:
        cell_boundaries (Shapes)
    ▸ 'pixels', with elements:
        _20221020095443.ome (Images), ARMS_Marks (Points), tiles (Shapes)

In [2]:
import scanpy as sc
Tissue = "Region2"
adata = sc.read_h5ad("/Volumes/ProstateCancerEvoMain/dbs/Ongoing/Region2/REGION2_TABLES/Region2_Xenium_Phen_HE_Integrated.Xenium_Process_Table.Cell_Type_Annotated_Auto.V3.All_Genes.h5ad")

sdata.shapes["cell_boundaries"].index = adata.obs["cell_id"].values

In [3]:
import pandas as pd

around_cells = pd.read_csv("/Users/ugursahin/ProstateCancerEvo_SpatialAnalysis/scripts/genomics_integration/Tiles_In_R2.csv")
tile_names = around_cells["name"].to_list()

In [4]:
sdata.shapes["tiles"]["color_joined"] = sdata.shapes["tiles"]["color"].apply(lambda x: "_".join(map(str, x)))

In [ ]:
from spatialdata import polygon_query

## Run for all
collectedTiles = dict()
for idx in range(sdata.shapes["tiles"].shape[0]):
    try:
        poi, name = sdata["tiles"].geometry.iloc[idx], sdata["tiles"].name.iloc[idx]
        if name in tile_names:
            col = sdata["tiles"].color.iloc[idx].astype(str)
            color = col[0]+"_"+col[1]+"_"+col[2]

            collectedTiles[name+"__col_"+color] =  polygon_query(sdata, polygon=poi, target_coordinate_system="aligned").shapes["cell_boundaries"].index.to_list()

            print(idx, name, color)
        else:
            print(f"{name} is skipped, NOT IN R2!")
    except:
        print(f"{name} is skipped, set is EMPTY ! ")
    

0 plate4_7B 255_255_255
1 plate4_4A 255_255_0
2 plate4_11E 102_0_102
3 plate4_2D 102_0_102
4 plate4_12G 102_0_102
5 plate4_5A 102_0_102
6 plate4_5A 102_0_102
plate4_9H is skipped, NOT IN R2!
8 plate4_10H 102_0_102
9 plate4_8B 255_255_255
plate4_1D is skipped, set is EMPTY ! 
11 plate4_11E 102_0_102
12 plate4_3D 128_0_128
plate4_7D is skipped, set is EMPTY ! 
plate4_12B is skipped, NOT IN R2!
15 plate4_10H 102_0_102
plate4_6D is skipped, NOT IN R2!
17 plate4_7C 255_255_0
18 plate4_1D 102_0_102
19 plate4_5D 255_255_0
20 plate4_10C 102_0_102
21 plate4_1B 255_255_0
22 plate4_7B 255_255_255
23 plate4_12F 102_0_102
24 plate4_11H 102_0_102
25 plate4_12A 102_0_102
26 plate4_5C 102_0_102
27 plate4_12D 102_0_102
plate4_9C is skipped, NOT IN R2!
29 plate4_8C 102_0_102
30 plate4_12E 255_255_0
31 plate4_3E 102_0_102
plate4_7H is skipped, NOT IN R2!
33 plate4_1G 255_255_0
34 plate4_2E 102_0_102
35 plate4_8G 0_255_0
36 plate4_9E 102_0_102
plate4_1H is skipped, NOT IN R2!
38 plate4_5H 102_0_102
39 pla

In [5]:
# import napari_spatialdata
# napari_spatialdata.Interactive(sdata)

In [ ]:
import pandas as pd

rows = [(k, v_i) for k, v in collectedTiles.items() for v_i in v]

barcode_key_paired_df = pd.DataFrame(rows, columns=["key", "value"])


# First, ensure the 'key' column is string type
barcode_key_paired_df["key"] = barcode_key_paired_df["key"].astype(str)

# Split the key column into two parts using '_col_' as the separator
split_df = barcode_key_paired_df["key"].str.split("_col_", expand=True)

# Assign the new columns
split_df.columns = ["Tile", "Clone"]

# Create the final DataFrame
final_df = pd.DataFrame({
    "cell_id": barcode_key_paired_df["value"],         # or use barcode_key_paired_df["some_id_column"] if index is not cell_id
    "Tile": split_df["Tile"],
    "Clone": split_df["Clone"],
})

final_df.to_csv("Barcode_TileClone_Pair.csv", index=False)

In [24]:
tiles_shapes = sdata.shapes["tiles"]["color"].apply(lambda x: "_".join(map(str, x))).copy()
sdata.shapes["tiles"]["color_joined"] = tiles_shapes

In [25]:
from collections import Counter
Counter(sdata.shapes["tiles"]["color_joined"])

Counter({'0_255_255': 1719,
         '102_0_102': 269,
         '255_255_0': 53,
         '255_255_255': 28,
         '0_255_0': 24,
         '128_0_128': 12,
         '0_0_0': 4})

In [ ]:
import napari_spatialdata

# sdata.shapes["tiles"]["color_joined"] = (
#     sdata.shapes["tiles"]["color_joined"]
#     .apply(lambda c: c if c == "0_0_0" else "red")
# )

napari_spatialdata.Interactive(sdata)

2025-06-05 00:21:55.020 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-06-05 00:21:55.022 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
Traceback (most recent call last):
  File "/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/napari/utils/action_manager.py", line 229, in _trigger
    self.trigger(name)
  File "/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/napari/utils/action_manager.py", line 431, in trigger
    return self._actions[name].injected()
  File "/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/in_n_out/_store.py", line 804, in _exec
    result = func(**bound.arguments)
  File "/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/napari/components/_viewer_key_bindings.py", line 100, in delete_selected_layers
    viewer.layers.remove_selected()
  File "/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/napari/utils/_proxies

In [ ]:
# #poi = sdata["tiles"].geometry.iloc[0]
# poi = sdata["POI"].geometry.iloc[0]
# sdata_tile_oi = polygon_query(sdata, polygon=poi, target_coordinate_system="aligned")

# len(polygon_query(sdata, polygon=sdata.shapes["POI_2"].geometry.iloc[4], target_coordinate_system="aligned").shapes["cell_boundaries"].index.to_list())
# len(polygon_query(sdata, polygon=sdata.shapes["tiles"].geometry.iloc[0], target_coordinate_system="aligned").shapes["cell_boundaries"].index.to_list())

# sdata.shapes["tiles"] = sdata.shapes["tiles"].iloc[[0,1,2,3,4]]

In [33]:
# transf = spatialdata.transformations.get_transformation_between_coordinate_systems(sdata, 'pixels', 'global', intermediate_coordinate_systems=sdata.images['_20221020095443.ome'])
# transf